# Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import inflection
import umap
import re
import pickle 
import sqlite3
import boto3
import sys
from io import StringIO
# import psycopg2


from sqlalchemy import create_engine


from sklearn.cluster         import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.ensemble        import RandomForestRegressor

## Loading

In [2]:
# loading via S3
client = boto3.client('s3')

bucket_name = 'pa005-dataset'
object_key = 'ecommerce.csv'

csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('iso-8859-1')

data = pd.read_csv(StringIO(csv_string))

In [3]:
#  Loading local
# path =  '/home/jonas/Documentos/repos/insiders_clustering/'

# data = pd.read_csv(path_s3, encoding='unicode_escape')

In [4]:
# drop de coluna
data = data.drop('Unnamed: 8', axis=1)

# 1 Descrição dos Dados

## 1.1 Renomeando as Colunas

In [5]:
old_cols = list(data.columns)

new_cols = []

for c in old_cols:
    new_cols.append(inflection.underscore(c))

data.columns = new_cols

## 1.5 Replace NA's

In [6]:
# criação de um dicionário com os novos customer_id {'invoice_no': novo_customer_id}
invoice_customer_id_null = list(data[data['customer_id'].isna()]['invoice_no'].drop_duplicates())

customer_id_null_dic = {}
new_customer_id = 18500
for invoice in invoice_customer_id_null:
    customer_id_null_dic[invoice] = new_customer_id
    new_customer_id += 1


In [7]:
# Nova coluna de customer_id, com invoice_no no lugar do customer_id
data['new_customer_id'] = data[['invoice_no', 'customer_id']].apply(lambda x: x['invoice_no'] if 
                                          np.isnan(x['customer_id']) == True else np.nan, axis=1)

In [8]:
# faz a troca de cada invoice_no pelo novo customer_id com o dicionário criado
data['new_customer_id'] = data['new_customer_id'].map(customer_id_null_dic)

In [9]:
# junta as colunas customer_id e new_customer_id em uma só
data['customer_id'] = data[['customer_id', 'new_customer_id']].apply(lambda x: x['new_customer_id'] 
                                                                     if np.isnan(x['customer_id'])  == True 
                                                                     else x['customer_id'] , axis=1)

In [10]:
# drop new_customer_id
data = data.drop('new_customer_id', axis=1)

## 1.6 Troca dos Tipos dos Dados

In [11]:
# invoice_date
data['invoice_date'] = pd.to_datetime(data['invoice_date'], format='%d-%b-%y')

# customer_id
data['customer_id'] = data['customer_id'].astype('int64')


# 2 Filtragem de Variáveis

In [12]:
df2 = data.copy()

In [13]:
# Unit_price: maiores que 0.04
df2 = df2[df2['unit_price'] > 0.04]

# stock_code: drop 'POST', 'C2', 'BANK CHARGES', 'PADS', 'DOT', 'CRUK'
df2 = df2[~df2['stock_code'].isin(['POST', 'C2', 'BANK CHARGES', 'PADS', 'DOT', 'CRUK', 'S', 'AMAZONFEE', 'B'])]

# description: drop da coluna
df2 = df2.drop('description', axis=1)

# customer outlier (analise relizada na seção 4)

df2 = df2[~df2['customer_id'].isin([12346, 14646])]

# colocar apenas os customers que tinham customer_id
df2 = df2[df2['customer_id'] < 18500]

# country: drop de 'European Community' e 'Unspecified' (idéia para os próximos ciclos)
# df2 = df2[~df2['country'].isin(['European Community', 'Unspecified'])]

# quantity
df2_returns = df2[df2['quantity'] < 0].copy()
df2_purchases = df2[df2['quantity'] >= 0].copy()



# 3 Features Engineering

In [14]:
df3 = df2.copy()

In [15]:
df_ref = df3.drop(['invoice_no', 'stock_code', 'quantity', 'invoice_date', 'unit_price', 'country'], axis=1)
df_ref = df_ref.drop_duplicates().reset_index(drop=True)

## 3.1 Criação de Features

### 3.1.1 Faturamento

In [16]:
# Monetary Purchase 
df2_purchases['monetary'] = df2_purchases['unit_price']*df2_purchases['quantity']
df_monetary_purchase = df2_purchases[['customer_id', 'monetary']].groupby('customer_id').sum().reset_index()

# join
df_ref = df_ref.merge(df_monetary_purchase, how='left', on='customer_id')

# Monetary Return
df2_returns['monetary'] = df2_returns['unit_price']*df2_returns['quantity']
df_monetary_returns = df2_returns[['customer_id', 'monetary']].groupby('customer_id').sum().reset_index()

# join
df_ref = df_ref.merge(df_monetary_returns, how='left', on='customer_id')

df_ref.rename(columns={'monetary_x':'monetary_purchase', 'monetary_y':'monetary_returns'}, inplace=True)

df_ref['monetary_returns'] = df_ref['monetary_returns'].fillna(0)

### 3.1.2 Recência  (Quantidade de dias desde de última compra)

In [17]:
# Recency (Recência) - Quantidade de dias desde a última compra
df2_purchases['recency_days'] = df2_purchases['invoice_date'].max() - df2_purchases['invoice_date']
df_recency = df2_purchases[['customer_id', 'recency_days']].groupby('customer_id').min().reset_index()
df_recency['recency_days'] = df_recency['recency_days'].dt.days     # extrai o dia como int64

# join
df_ref = df_ref.merge(df_recency, how='left', on='customer_id')


### 3.1.3 Quantidade de compras realizadas

In [18]:
# Frequency
df_aux = df2_purchases[['customer_id', 'invoice_no']].drop_duplicates().groupby('customer_id').count().reset_index()
df_aux.columns = ['customer_id', 'quant_invoices']

# Join
df_ref = df_ref.merge(df_aux, how='left', on='customer_id')

### 3.1.4 Quantidade de produtos comprados

In [19]:
df_aux = df2_purchases[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index()
df_aux.columns = ['customer_id', 'quant_items']

# Join
df_ref = df_ref.merge(df_aux, how='left', on='customer_id')

### 3.1.5 Quantidade de produtos únicos comprados

In [20]:
df_aux = df2_purchases[['customer_id', 'stock_code']].groupby('customer_id').count().reset_index()
df_aux.columns = ['customer_id', 'quant_prod_uniq']

# Join
df_ref = df_ref.merge(df_aux, how='left', on='customer_id')

### 3.1.6 Ticket Médio

In [21]:
# AVG Ticket
df2['monetary'] = df2['unit_price']*df2['quantity']
df_avg_ticket = df2[['customer_id', 'monetary']].groupby('customer_id').mean().reset_index().rename(columns={'monetary': 'avg_ticket'})

# Join
df_ref = df_ref.merge(df_avg_ticket, how='left', on='customer_id')

### 3.1.7 Recência média entre as compras

In [22]:
df_aux = df2_purchases[['customer_id', 'invoice_date']].drop_duplicates().sort_values(['customer_id', 'invoice_date'], ascending=[False, False])

df_aux['next_customer'] = df_aux['customer_id'].shift()
df_aux['next_date'] = df_aux['invoice_date'].shift()

data_max = df_aux['invoice_date'].max()
data_min = df_aux['invoice_date'].min()

df_aux['avg_recency_day'] = df_aux.apply(lambda x: (x['next_date'] - x['invoice_date']).days if x['customer_id'] == x['next_customer'] 
                                         else (data_max - data_min).days, axis=1)

df_aux = df_aux.drop(['invoice_date', 'next_customer', 'next_date'], axis=1)

df_avg_recency = df_aux[['customer_id', 'avg_recency_day']].groupby('customer_id').mean().reset_index()

# Join
df_ref = df_ref.merge(df_avg_recency, how='left', on='customer_id')

### 3.1.8 Frequência de Compras

In [23]:
df_aux = (df2_purchases[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates().groupby('customer_id')
                                                                              .agg(max_ = ('invoice_date', 'max'),
                                                                                   min_ = ('invoice_date', 'min'),
                                                                                   days_= ('invoice_date', lambda x: (x.max() - x.min()).days),
                                                                                   buy_ = ('invoice_no', 'count'))).reset_index()

df_aux['freq_purchases'] = df_aux.apply(lambda x: (x['buy_'] / x['days_']) if x['days_'] !=  0 else 0, axis=1)

df_freq_pur = df_aux[['customer_id', 'freq_purchases']]


# Join
df_ref = df_ref.merge(df_freq_pur, how='left', on='customer_id')

### 3.1.9 Número de Retornos

In [24]:
df_returns = df2_returns[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity':'quant_returns'})

df_returns['quant_returns'] = df_returns['quant_returns'] * -1

# Join
df_ref = df_ref.merge(df_returns, how='left', on='customer_id')

# Fillna
df_ref['quant_returns'] = df_ref['quant_returns'].fillna(0)

### 3.1.10 Média da quantidade de produtos por compra

In [25]:
df_aux = (df2_purchases[['customer_id', 'invoice_no', 'quantity']].groupby('customer_id')
                                                                  .agg(n_purchases = ('invoice_no', 'nunique'),
                                                                       n_product = ('quantity', 'sum'))
                                                                  .reset_index())

df_aux['avg_basket_size'] = df_aux['n_product'] / df_aux['n_purchases']

df_avg_basket = df_aux[['customer_id', 'avg_basket_size']]


# Join
df_ref = df_ref.merge(df_avg_basket, how='left', on='customer_id')

### 3.1.11 Média da quantidade de produtos únicos por compra

In [26]:
df_aux = (df2_purchases[['customer_id', 'invoice_no', 'stock_code']].groupby('customer_id')
                                                         .agg(n_purchases = ('invoice_no', 'nunique'),
                                                             n_product = ('stock_code', 'nunique'))
                                                         .reset_index())


df_aux['avg_uniq_basket_size'] = df_aux['n_product'] / df_aux['n_purchases']

df_avg_uniq_basket = df_aux[['customer_id', 'avg_uniq_basket_size']]

# Join
df_ref = df_ref.merge(df_avg_uniq_basket, how='left', on='customer_id')

# 4 Análise Exploratória dos Dados

In [27]:
df4 = df_ref.dropna().reset_index(drop=True).copy()

## 4.3 Estudo dos Espaços

In [28]:
# drop de um cliente
df4 = df4[df4['customer_id'] != 16446].reset_index(drop=True)


### 4.3.4 Embedding baseado em árvores

In [29]:
X = df4.drop(['customer_id', 'monetary_purchase'], axis=1).copy()
y = df4['monetary_purchase'].copy()

In [30]:
# model defition
rf_model = RandomForestRegressor(n_estimators=2000,  random_state=42)

# fit 
rf_model.fit(X, y)

# Leaf
leaf = rf_model.apply(X)  # retorna os índices das folhas

# dataframe leaf
df_leaf = pd.DataFrame(leaf)

In [31]:
reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(df_leaf)

# embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[: , 0]
df_tree['embedding_y'] = embedding[: , 1]

# 5 Preparação dos Dados

In [32]:
df5 = df_tree.copy()
# df5.to_csv('df_tree_embedding.csv', index=False)

# 6 Seleção dos Atributos

In [33]:
df6 = df5.copy()

Todos os atributos foram selecionados.

In [34]:
df7 = df6.copy()

# 8 Treinamento do Modelo

In [35]:
df8 = df7.copy()
X = df8.copy()

# 9 Análise dos Clusters

In [36]:
df9 = df8.copy()

# load model
# model = pickle.load(open('/home/jonas/Documentos/repos/insiders_clustering/model/hc_model.pkl', 'rb'))
k = 10
model = AgglomerativeClustering(n_clusters=k)

labels  = model.fit(X)

df9['clusters'] = model.labels_

## 9.1 Inspeção por Visualização

Após analisar diversas possibilidades, optei por utilizar o modelo de **HC** com **10 clusters**.

## 9.2 Perfil dos Clusters

In [37]:
df92 = df4.copy()
df92['clusters'] = df9['clusters']

In [38]:
# trocando os tipos das variáveis
df92['recency_days'] = df92['recency_days'].astype(int)
df92['quant_invoices'] = df92['quant_invoices'].astype(int)
df92['quant_items'] = df92['quant_items'].astype(int)
df92['quant_prod_uniq'] = df92['quant_prod_uniq'].astype(int)
df92['quant_returns'] = df92['quant_returns'].astype(int)

In [39]:
df92.head()

,customer_id,monetary_purchase,monetary_returns,recency_days,quant_invoices,quant_items,quant_prod_uniq,avg_ticket,avg_recency_day,freq_purchases,quant_returns,avg_basket_size,avg_uniq_basket_size,clusters
0,17850,5391.21,-102.58,372,34,1733,297,16.950737,187.000000,34.000000,40,50.970588,0.617647,6
1,13047,3237.54,-158.44,31,10,1391,172,15.709694,89.375000,0.029240,36,139.100000,10.600000,6
2,12583,6705.38,-76.04,2,15,5028,232,28.209957,49.600000,0.040431,50,335.200000,7.600000,2
3,13748,948.25,0.00,95,5,439,28,33.866071,162.750000,0.017986,0,87.800000,4.800000,0
4,15100,876.00,-240.90,333,3,80,3,105.850000,137.666667,0.075000,22,26.666667,0.333333,5


In [40]:
# Number of Clusters
df_clusters = df92[['customer_id', 'clusters']].groupby('clusters').count().reset_index()
df_clusters['customer_perc'] = 100*(df_clusters['customer_id'] / df_clusters['customer_id'].sum())

# AVG Monetary
df_avg_monetary_p = df92[['monetary_purchase', 'clusters']].groupby('clusters').mean().reset_index()
df_clusters = df_clusters.merge(df_avg_monetary_p, on='clusters')

# AVG Monetary_Returns
df_avg_monetary_r = df92[['monetary_returns', 'clusters']].groupby('clusters').mean().reset_index()
df_clusters = df_clusters.merge(df_avg_monetary_r, on='clusters')

# AVG Recency_days
df_avg_recency = df92[['recency_days', 'clusters']].groupby('clusters').mean().reset_index()
df_clusters = df_clusters.merge(df_avg_recency, on='clusters')

# AVG quant_invoices
df_avg_invoices = df92[['quant_invoices', 'clusters']].groupby('clusters').mean().reset_index()
df_clusters = df_clusters.merge(df_avg_invoices, on='clusters')

# AVG quant_items
df_avg_items = df92[['quant_items', 'clusters']].groupby('clusters').mean().reset_index()
df_clusters = df_clusters.merge(df_avg_items, on='clusters')

# AVG quant_prod_uniq
df_avg_prod_uniq = df92[['quant_prod_uniq', 'clusters']].groupby('clusters').mean().reset_index()
df_clusters = df_clusters.merge(df_avg_prod_uniq, on='clusters')

# AVG avg_ticket
df_avg_avg_ticket = df92[['avg_ticket', 'clusters']].groupby('clusters').mean().reset_index()
df_clusters = df_clusters.merge(df_avg_avg_ticket, on='clusters')

# AVG avg_recency_day

# AVG Frequency
df_avg_freq = df92[['freq_purchases', 'clusters']].groupby('clusters').mean().reset_index()
df_clusters = df_clusters.merge(df_avg_freq, on='clusters')

# AVG quant_returns
df_avg_returns = df92[['quant_returns', 'clusters']].groupby('clusters').mean().reset_index()
df_clusters = df_clusters.merge(df_avg_returns, on='clusters')

# Rename columns
df_clusters.columns = ['clusters', 'num_customer', 'customer_perc', 'monetary_avg_p', 'monetary_avg_r', 'recency_days_avg', 'quant_invoices_avg', 'quant_items_avg', 'quant_prod_uniq_avg', 'avg_ticket_avg', 'frequency_avg', 'quant_returns_avg']

# 10 Análise Exploratória dos Dados

In [41]:
df10 = df92.copy()

# 11 Deploy do Modelo em Produção

## Inserindo no SQLite

In [42]:
#criando a tabela
# query_create_table_insiders = """
#     CREATE TABLE insiders(
#         customer_id            INTEGER,     
#         monetary_purchase      REAL,
#         monetary_returns       REAL,
#         recency_days           INTEGER, 
#         quant_invoices         INTEGER, 
#         quant_items            INTEGER, 
#         quant_prod_uniq        INTEGER, 
#         avg_ticket             REAl, 
#         avg_recency_day        REAl, 
#         freq_purchases         REAl, 
#         quant_returns          INTEGER, 
#         avg_basket_size        REAl, 
#         avg_uniq_basket_size   REAl, 
#         clusters               INTEGER 
# )
# """

# # criando o banco de dados
# conn = sqlite3.connect('insiders_db.sqlite')
# conn.execute(query_create_table_insiders)
# conn.commit()
# conn.close()

# insert data
# conn = create_engine('sqlite:///insiders_db.sqlite')
# df92.to_sql('insiders', con = conn, if_exists='replace', index=False)


4334

## Inserindo no RDS da AWS

In [43]:
user = 'jonas'
pwd = 'insiders123!'
host = 'database-pa005.c2gxtj3tua5w.us-east-1.rds.amazonaws.com'
port = '5432'
database = 'postgres'
endpoint = f'postgresql://{user}:{pwd}@{host}:{port}/{database}'

conn = create_engine(endpoint)

In [44]:
# # criando a tabela
# query_create_table_insiders = """
#     CREATE TABLE insiders(
#         customer_id            INTEGER,     
#         monetary_purchase      REAL,
#         monetary_returns       REAL,
#         recency_days           INTEGER, 
#         quant_invoices         INTEGER, 
#         quant_items            INTEGER, 
#         quant_prod_uniq        INTEGER, 
#         avg_ticket             REAl, 
#         avg_recency_day        REAl, 
#         freq_purchases         REAl, 
#         quant_returns          INTEGER, 
#         avg_basket_size        REAl, 
#         avg_uniq_basket_size   REAl, 
#         clusters               INTEGER 
# )
# """

# conn.execute(query_create_table_insiders)

NameError: name 'query_create_table_insiders' is not defined

In [45]:
df92.to_sql('insiders', con = conn, if_exists='replace', index=False)

334

In [46]:
# consultando database
query = """
SELECT *
FROM insiders
"""

df = pd.read_sql_query(query, conn)

conn.close()

In [48]:
df.shape

(4334, 14)

In [ ]:
df.to_csv('datasets/customers.csv')